
# 🐍 04 — Agents Hands-On in Python (Option A)

This notebook gives you **practical, code-focused building blocks** for Agentic AI in Python:

- Implementing a minimal **ReAct-style agent loop**
- Defining **tools** as Python callables with schemas
- Adding **memory** and simple **planning**
- Sketching a **multi-agent** pattern
- Showing how this connects to **RAG** and **MCP** later



## 1. Setup

We’ll assume:

- Python 3.10+
- An `openai`-compatible client (can be OpenAI, Azure, or a local shim)
- Environment variable `OPENAI_API_KEY` (or equivalent) is set

For now, all calls are structured but we keep actual LLM calls **minimal and pluggable**.


In [ ]:
import os
from dataclasses import dataclass, field
from typing import Any, Callable, Dict, List, Optional, Tuple

# If you have an OpenAI-compatible client, you can adapt this later.
# For now, we leave a placeholder function for "llm_complete" to keep this notebook self-contained.

def llm_complete(system_prompt: str, messages: List[Dict[str, str]]) -> str:
    """Placeholder LLM completion.

    Replace this with a real call, for example:

    from openai import OpenAI
    client = OpenAI()
    resp = client.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[{"role": "system", "content": system_prompt}, *messages],
    )
    return resp.choices[0].message.content
    """
    # For now, just echo a simple string so the loop is testable without a real LLM.
    return "THOUGHT: I will call the 'echo' tool.
ACTION: echo
ACTION_INPUT: {"text": "Hello from mock LLM"}"



## 2. A Minimal Tool Abstraction

We define a simple **Tool** concept:

- `name`
- `description`
- `func(input_dict) -> output_dict`
- optional `schema` (for documentation/validation later)


In [ ]:
@dataclass
class Tool:
    name: str
    description: str
    func: Callable[[Dict[str, Any]], Dict[str, Any]]

    def __call__(self, tool_input: Dict[str, Any]) -> Dict[str, Any]:
        return self.func(tool_input)


class ToolRegistry:
    def __init__(self):
        self._tools: Dict[str, Tool] = {}

    def register(self, tool: Tool):
        if tool.name in self._tools:
            raise ValueError(f"Tool '{tool.name}' is already registered.")
        self._tools[tool.name] = tool

    def get(self, name: str) -> Optional[Tool]:
        return self._tools.get(name)

    def list_tools(self) -> List[Tool]:
        return list(self._tools.values())



## 3. Example Tools

We’ll define some simple tools:

- `echo` – echoes text back
- `add_numbers` – adds two numbers
- `search_notes` – fake search over an in-memory list (stand-in for RAG)


In [ ]:
# In-memory "notes" for a fake search tool
NOTES = [
    {"id": 1, "content": "Python agents can use tools and memory."},
    {"id": 2, "content": "RAG combines retrieval with generation."},
    {"id": 3, "content": "MCP provides a capability layer via tools."},
]

def echo_tool(input_dict: Dict[str, Any]) -> Dict[str, Any]:
    text = str(input_dict.get("text", ""))
    return {"text": text}

def add_numbers_tool(input_dict: Dict[str, Any]) -> Dict[str, Any]:
    a = float(input_dict.get("a", 0))
    b = float(input_dict.get("b", 0))
    return {"result": a + b}

def search_notes_tool(input_dict: Dict[str, Any]) -> Dict[str, Any]:
    query = str(input_dict.get("query", "")).lower()
    matches = [n for n in NOTES if query in n["content"].lower()]
    return {"matches": matches}

registry = ToolRegistry()
registry.register(Tool(name="echo", description="Echo back text", func=echo_tool))
registry.register(Tool(name="add_numbers", description="Add two numbers", func=add_numbers_tool))
registry.register(Tool(name="search_notes", description="Search in-memory notes", func=search_notes_tool))

[ (t.name, t.description) for t in registry.list_tools() ]



## 4. A Minimal ReAct-Style Agent Loop

We implement a **single-agent loop**:

- The agent receives a user query
- It keeps a **history** of:
  - user messages
  - thoughts
  - tool calls
  - observations
- The LLM outputs:
  - `THOUGHT: ...`
  - `ACTION: tool_name` (or `NONE`)
  - `ACTION_INPUT: {...}` (JSON)
- The agent executes the tool (if any), then feeds observations back to the LLM.


In [ ]:
import json
import re

REACT_SYSTEM_PROMPT = """You are a helpful agent that can use tools.

You will respond using this format:

THOUGHT: <your reasoning>
ACTION: <tool_name or NONE>
ACTION_INPUT: <JSON object with arguments>

Available tools will be described to you. If no tool is needed, set ACTION to NONE.
"""


def format_tool_list(tools: List[Tool]) -> str:
    lines = []
    for t in tools:
        lines.append(f"- {t.name}: {t.description}")
    return "\n".join(lines)


def parse_react_response(text: str) -> Tuple[str, str, Dict[str, Any]]:
    thought_match = re.search(r"THOUGHT:(.*)", text)
    action_match = re.search(r"ACTION:(.*)", text)
    input_match = re.search(r"ACTION_INPUT:(.*)", text, re.DOTALL)

    thought = thought_match.group(1).strip() if thought_match else ""
    action = action_match.group(1).strip() if action_match else "NONE"
    raw_input = input_match.group(1).strip() if input_match else "{}"

    try:
        action_input = json.loads(raw_input)
    except json.JSONDecodeError:
        action_input = {"raw": raw_input, "parse_error": True}

    return thought, action, action_input


@dataclass
class AgentStep:
    user: Optional[str] = None
    thought: Optional[str] = None
    action: Optional[str] = None
    action_input: Optional[Dict[str, Any]] = None
    observation: Optional[Dict[str, Any]] = None


@dataclass
class SimpleAgent:
    tools: ToolRegistry
    max_steps: int = 5
    memory: List[AgentStep] = field(default_factory=list)

    def run(self, user_query: str) -> str:
        """Run a ReAct-style loop until the agent stops or hits step limit."""
        self.memory.clear()
        self.memory.append(AgentStep(user=user_query))

        for step_idx in range(self.max_steps):
            messages = self._build_messages(user_query)
            tool_list_str = format_tool_list(self.tools.list_tools())
            messages.append({
                "role": "user",
                "content": f"User question: {user_query}\n\nAvailable tools:\n{tool_list_str}",
            })

            raw = llm_complete(REACT_SYSTEM_PROMPT, messages)
            thought, action, action_input = parse_react_response(raw)

            current_step = AgentStep(
                user=user_query if step_idx == 0 else None,
                thought=thought,
                action=action,
                action_input=action_input,
            )

            if action.upper() == "NONE":
                # Final answer is the thought or separate final message if you want to extend format
                self.memory.append(current_step)
                return thought or raw

            tool = self.tools.get(action)
            if not tool:
                current_step.observation = {"error": f"Unknown tool '{action}'"}
                self.memory.append(current_step)
                return f"Error: unknown tool '{action}'."

            obs = tool(action_input or {})
            current_step.observation = obs
            self.memory.append(current_step)

        return "I reached the maximum number of steps without finishing."

    def _build_messages(self, user_query: str) -> List[Dict[str, str]]:
        messages: List[Dict[str, str]] = []
        # You can add a compressed memory summary here in a more advanced version.
        for s in self.memory:
            if s.user:
                messages.append({"role": "user", "content": s.user})
            if s.thought:
                messages.append({"role": "assistant", "content": f"THOUGHT: {s.thought}"})
            if s.action:
                messages.append({"role": "assistant", "content": f"ACTION: {s.action}"})
            if s.observation is not None:
                messages.append({"role": "user", "content": f"OBSERVATION: {s.observation}"})
        return messages



### Test the SimpleAgent with Mocked LLM

Remember, we’re using a **mock** `llm_complete` that always calls the `echo` tool.  
You can later swap `llm_complete` with a real client.


In [ ]:
agent = SimpleAgent(tools=registry, max_steps=3)
answer = agent.run("Say hi using the echo tool.")
answer, agent.memory



## 5. Adding Simple Memory

We now extend the agent with:

- A **conversation memory** summary (simple version: concatenation)
- The idea that memory can influence future reasoning


In [ ]:
@dataclass
class MemoryAwareAgent(SimpleAgent):
    def _summarize_memory(self) -> str:
        # Very naive memory summarization
        parts = []
        for idx, s in enumerate(self.memory):
            if s.user:
                parts.append(f"[User] {s.user}")
            if s.thought:
                parts.append(f"[Thought {idx}] {s.thought}")
            if s.action:
                parts.append(f"[Action {idx}] {s.action} {s.action_input}")
            if s.observation is not None:
                parts.append(f"[Obs {idx}] {s.observation}")
        return "\n".join(parts)

    def _build_messages(self, user_query: str) -> List[Dict[str, str]]:
        messages: List[Dict[str, str]] = []
        if self.memory:
            summary = self._summarize_memory()
            messages.append({
                "role": "system",
                "content": f"Conversation summary so far:\n{summary}",
            })
        return messages

mem_agent = MemoryAwareAgent(tools=registry, max_steps=3)
answer2 = mem_agent.run("Search notes about RAG and tell me what you find.")
answer2



> 🔎 **Note:**  
> This is intentionally simple. In a production-grade system you would:
> - Use vector stores for long-term memory
> - Use summarization models for compression
> - Store memory externally (DB, Redis, etc.)



## 6. Sketching a Multi-Agent Pattern (Manager → Worker)

Here we sketch how a **manager agent** could delegate to a **worker agent**.

We keep this minimal and focus on structure, not perfect LLM logic.


In [ ]:
@dataclass
class WorkerAgent(SimpleAgent):
    role: str = "worker"

    def run_task(self, task: str) -> str:
        return super().run(task)


@dataclass
class ManagerAgent(SimpleAgent):
    worker: WorkerAgent = None  # type: ignore

    def run(self, user_query: str) -> str:
        # Very naive: manager decomposes into a single sub-task for the worker.
        sub_task = f"Perform the main work for: {user_query}"
        worker_result = self.worker.run_task(sub_task)
        # Manager then wraps worker result as final answer.
        return f"As manager, I delegated the work. Worker result: {worker_result}"


worker = WorkerAgent(tools=registry, max_steps=3)
manager = ManagerAgent(tools=registry, max_steps=2, worker=worker)

manager_answer = manager.run("Help me understand how agents use tools.")
manager_answer



## 7. How This Connects to RAG and MCP

This notebook focuses on *pure Python agents*, but in your full ecosystem:

- **RAG Universe** provides:
  - Retrieval tools (search, get_document, etc.)
  - Evaluation tools (RAGAS, LLM-as-judge)

- **MCP Universe** provides:
  - A tool / capability layer for:
    - filesystem
    - HTTP
    - DB
    - RAG servers
    - DevOps & Finance tools

- **Agents Universe (this one)**:
  - Uses those tools via:
    - ReAct-style loops
    - Planner–executor patterns
    - Multi-agent systems

In later notebooks (08, 09, 10), you will:

- Turn the **fake `search_notes` tool** into real RAG calls  
- Turn the **simple `Tool` abstraction** into MCP tool calls  
- Implement full **Agentic RAG** and **Agents + MCP** systems.
